In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#create the grouped dataset based on the data from each participant
vlad_file = open("/content/drive/MyDrive/NLP/datavlad.txt")
vlad_sentences = vlad_file.read().split('.')[:-1]

artem_file = open("/content/drive/MyDrive/NLP/dataartem.txt")
artem_sentences = artem_file.read().split('.')[:-1]

ks_file = open("/content/drive/MyDrive/NLP/dataks.txt")
ks_sentences = ks_file.read().split('.')[:-1]

In [4]:
new_df = []

for new in vlad_sentences:
  new_df.append({'Sentence':new, 'Label':'neural network art'})

for new in ks_sentences:
  new_df.append({'Sentence':new, 'Label':'climate change'})

for new in artem_sentences:
  new_df.append({'Sentence':new, 'Label':'neural network recognition'})

new_df = pd.DataFrame(data=new_df, columns=['Sentence', 'Label'])

new_df['Target'] = new_df['Label']
new_df.replace({'Target':{'neural network art':1, 'climate change':0, 'neural network recognition':0}}, inplace=True)
new_df

,Sentence,Label,Target
0,"In the past few years, many artists have begun...",neural network art,1
1,\nIn computer vision and perceptual psychology...,neural network art,1
2,"\nIn other words, modern neural models lend th...",neural network art,1
3,\nThe most prominent tool in neural art at the...,neural network art,1
4,\nGiven a large collection of images of a spec...,neural network art,1
...,...,...,...
302,"\nIn May 2017, a man was arrested using an aut...",neural network recognition,0
303,[68] Live facial recognition has been trialled...,neural network recognition,0
304,[69] In August 2020 the Court of Appeal ruled ...,neural network recognition,0
305,S,neural network recognition,0


In [5]:
# vectorization of text
max_tokens = 10000

sentences = vlad_sentences+artem_sentences+ks_sentences
tokens_count = 0
for new in sentences:
  tokens_count+=len(new.split())
avg_tokens = round(tokens_count/len(sentences))

In [6]:
#tokenization and embedding
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(new_df['Sentence'])

text_vectorizer(new_df['Sentence'])

<tf.Tensor: shape=(307, 25), dtype=int64, numpy=
array([[   6,    2,  212, ...,  826,    6, 1800],
       [   6,  303,  328, ...,    0,    0,    0],
       [   6,   62,  325, ...,  139,  455,  266],
       ...,
       [2051,    6, 1943, ...,  251,  101,    6],
       [ 575,    0,    0, ...,    0,    0,    0],
       [ 763,    3,  363, ...,   24,  102,  524]])>

In [7]:
embedding = layers.Embedding(input_dim=max_tokens, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=avg_tokens) # how long is each input

embedding(text_vectorizer(new_df['Sentence']))

<tf.Tensor: shape=(307, 25, 128), dtype=float32, numpy=
array([[[-0.02110715,  0.02350539,  0.04923756, ..., -0.0399381 ,
          0.01319214, -0.01607747],
        [ 0.01041071, -0.0288753 , -0.01059166, ...,  0.03674814,
          0.04063675,  0.00346867],
        [-0.02473914,  0.00615771,  0.00389104, ...,  0.03205865,
          0.02993964,  0.03718005],
        ...,
        [-0.00638632,  0.00332669,  0.0345796 , ...,  0.01410979,
          0.01467253,  0.01910256],
        [-0.02110715,  0.02350539,  0.04923756, ..., -0.0399381 ,
          0.01319214, -0.01607747],
        [ 0.02502717,  0.04445639, -0.04494101, ...,  0.04625538,
         -0.01475898,  0.00666505]],

       [[-0.02110715,  0.02350539,  0.04923756, ..., -0.0399381 ,
          0.01319214, -0.01607747],
        [-0.04333587, -0.04216122, -0.04893682, ..., -0.03010637,
          0.02679728,  0.00208858],
        [ 0.04383955,  0.04991759,  0.04202527, ..., -0.00055216,
         -0.04173743, -0.02878193],
        ...

In [8]:
# import libraries
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(new_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Label,Target
197,Democrats also reported much higher confidenc...,climate change,0
81,\nWhile the first stage deep CNN classifier is...,neural network art,1
303,[68] Live facial recognition has been trialled...,neural network recognition,0
70,\nTransfer learning allows the adaptation or r...,neural network art,1
92,\nThis type of approach relies on the assumpti...,neural network art,1
...,...,...,...
95,"\nIn this paper, we explore the secret nature ...",neural network art,1
162,This process is known as the greenhouse effec...,climate change,0
77,\nThe findings revealed a strong correlation b...,neural network art,1
102,\nWe can see that our method successfully lear...,neural network art,1


In [9]:
#import libraries for classification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


In [10]:
#lstm
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_lstm = tf.keras.Model(inputs, outputs, name="model_LSTM")


In [11]:
model_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [12]:
model_lstm.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 25)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 25, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [13]:
model_lstm_history = model_lstm.fit(train_split['Sentence'],
                              train_split['Target'],
                              epochs=9,
                              validation_data=(test_split['Sentence'], test_split['Target']))


Epoch 1/9
8/8 [==============================] - 6s 109ms/step - loss: 0.6820 - accuracy: 0.6408 - val_loss: 0.6529 - val_accuracy: 0.6774
Epoch 2/9
8/8 [==============================] - 0s 17ms/step - loss: 0.6396 - accuracy: 0.6531 - val_loss: 0.5969 - val_accuracy: 0.6774
Epoch 3/9
8/8 [==============================] - 0s 17ms/step - loss: 0.5370 - accuracy: 0.6531 - val_loss: 0.4952 - val_accuracy: 0.6935
Epoch 4/9
8/8 [==============================] - 0s 17ms/step - loss: 0.3058 - accuracy: 0.8980 - val_loss: 0.2839 - val_accuracy: 0.9355
Epoch 5/9
8/8 [==============================] - 0s 19ms/step - loss: 0.1108 - accuracy: 0.9918 - val_loss: 0.3611 - val_accuracy: 0.8548
Epoch 6/9
8/8 [==============================] - 0s 18ms/step - loss: 0.0297 - accuracy: 0.9959 - val_loss: 0.4767 - val_accuracy: 0.8710
Epoch 7/9
8/8 [==============================] - 0s 17ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.4898 - val_accuracy: 0.9032
Epoch 8/9
8/8 [==================

In [14]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Target'], y_pred=model_lstm_preds)


{'accuracy': 90.32258064516128,
 'f1': 0.9043497808249972,
 'precision': 0.9074780058651026,
 'recall': 0.9032258064516129}

In [35]:
#GRU
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x) 
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_gru = tf.keras.Model(inputs, outputs, name="model_GRU")


In [36]:
model_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [37]:
model_gru_history = model_gru.fit(train_split['Sentence'],
                              train_split['Target'],
                              epochs=9,
                              validation_data=(test_split['Sentence'], test_split['Target']))


Epoch 1/9
8/8 [==============================] - 3s 90ms/step - loss: 0.6664 - accuracy: 0.6776 - val_loss: 0.6454 - val_accuracy: 0.7742
Epoch 2/9
8/8 [==============================] - 0s 16ms/step - loss: 0.5699 - accuracy: 0.8939 - val_loss: 0.5451 - val_accuracy: 0.7742
Epoch 3/9
8/8 [==============================] - 0s 17ms/step - loss: 0.3590 - accuracy: 0.9388 - val_loss: 0.3493 - val_accuracy: 0.8548
Epoch 4/9
8/8 [==============================] - 0s 16ms/step - loss: 0.0736 - accuracy: 0.9918 - val_loss: 0.2134 - val_accuracy: 0.9032
Epoch 5/9
8/8 [==============================] - 0s 16ms/step - loss: 0.0085 - accuracy: 0.9959 - val_loss: 0.4803 - val_accuracy: 0.9194
Epoch 6/9
8/8 [==============================] - 0s 17ms/step - loss: 2.0660e-04 - accuracy: 1.0000 - val_loss: 0.5081 - val_accuracy: 0.9355
Epoch 7/9
8/8 [==============================] - 0s 16ms/step - loss: 9.9250e-05 - accuracy: 1.0000 - val_loss: 0.5444 - val_accuracy: 0.9355
Epoch 8/9
8/8 [===========

In [38]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Target'], y_pred=model_gru_preds)

{'accuracy': 93.54838709677419,
 'f1': 0.9345362656453587,
 'precision': 0.9358911697621375,
 'recall': 0.9354838709677419}

In [19]:
#bidirectional LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_lstm = tf.keras.Model(inputs, outputs, name="model_Bi_lstm")


In [20]:
model_bi_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [21]:
model_bi_lstm.summary()


Model: "model_Bi_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 25)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 25, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [22]:
model_bi_lstm_history = model_bi_lstm.fit(train_split['Sentence'],
                              train_split['Target'],
                              epochs=9,
                              validation_data=(test_split['Sentence'], test_split['Target']))


Epoch 1/9
8/8 [==============================] - 5s 159ms/step - loss: 0.6447 - accuracy: 0.6939 - val_loss: 0.5853 - val_accuracy: 0.7581
Epoch 2/9
8/8 [==============================] - 0s 25ms/step - loss: 0.4139 - accuracy: 0.9143 - val_loss: 0.3579 - val_accuracy: 0.8226
Epoch 3/9
8/8 [==============================] - 0s 24ms/step - loss: 0.1215 - accuracy: 0.9755 - val_loss: 0.2186 - val_accuracy: 0.8871
Epoch 4/9
8/8 [==============================] - 0s 24ms/step - loss: 0.0157 - accuracy: 1.0000 - val_loss: 0.1906 - val_accuracy: 0.9516
Epoch 5/9
8/8 [==============================] - 0s 24ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.2573 - val_accuracy: 0.9355
Epoch 6/9
8/8 [==============================] - 0s 24ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.2915 - val_accuracy: 0.9355
Epoch 7/9
8/8 [==============================] - 0s 25ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.3761 - val_accuracy: 0.9032
Epoch 8/9
8/8 [==================

In [23]:
model_bi_lstm_pred_probs = model_bi_lstm.predict(test_split['Sentence'])
model_bi_lstm_preds = tf.squeeze(tf.round(model_bi_lstm_pred_probs))
calculate_results(y_true=test_split['Target'], y_pred=model_bi_lstm_preds)

{'accuracy': 91.93548387096774,
 'f1': 0.919852502061748,
 'precision': 0.9208722041137462,
 'recall': 0.9193548387096774}

In [24]:
#bidirectional GRU
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_gru = tf.keras.Model(inputs, outputs, name="model_Bi_gru")

In [25]:
model_bi_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [26]:
model_bi_gru.summary()

Model: "model_Bi_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 25)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 25, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,354,625
No

In [27]:
model_bi_gru_history = model_bi_gru.fit(train_split['Sentence'],
                              train_split['Target'],
                              epochs=9,
                              validation_data=(test_split['Sentence'], test_split['Target']))

Epoch 1/9
8/8 [==============================] - 5s 149ms/step - loss: 0.6417 - accuracy: 0.7102 - val_loss: 0.5849 - val_accuracy: 0.7419
Epoch 2/9
8/8 [==============================] - 0s 24ms/step - loss: 0.4806 - accuracy: 0.8327 - val_loss: 0.4683 - val_accuracy: 0.7903
Epoch 3/9
8/8 [==============================] - 0s 24ms/step - loss: 0.2816 - accuracy: 0.9020 - val_loss: 0.2887 - val_accuracy: 0.8710
Epoch 4/9
8/8 [==============================] - 0s 24ms/step - loss: 0.0602 - accuracy: 0.9959 - val_loss: 0.1128 - val_accuracy: 0.9516
Epoch 5/9
8/8 [==============================] - 0s 25ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1842 - val_accuracy: 0.9677
Epoch 6/9
8/8 [==============================] - 0s 24ms/step - loss: 2.9609e-04 - accuracy: 1.0000 - val_loss: 0.2932 - val_accuracy: 0.9516
Epoch 7/9
8/8 [==============================] - 0s 25ms/step - loss: 3.1300e-04 - accuracy: 1.0000 - val_loss: 0.3069 - val_accuracy: 0.9516
Epoch 8/9
8/8 [==========

In [28]:
model_bi_gru_pred_probs = model_bi_gru.predict(test_split['Sentence'])
model_bi_gru_preds = tf.squeeze(tf.round(model_bi_gru_pred_probs))
calculate_results(y_true=test_split['Target'], y_pred=model_bi_gru_preds)

{'accuracy': 95.16129032258065,
 'f1': 0.9512771858122903,
 'precision': 0.9515141943380583,
 'recall': 0.9516129032258065}

Every model has 9 epochs for learning. The best results were shown by bidirectional GRU model.